# Установка

In [ ]:
#@title # Установка (PIP, 3 минуты)
%cd /content/
!git clone https://github.com/noblebarkrr/mvsepless -b beta
!cd mvsepless && pip install -r requirements.txt
!cd mvsepless && pip install ./fixed/audio_separator-0.32.0-py3-none-any.whl

In [ ]:
#@title # Установка (UV, 1 минута)
%cd /content/
!git clone https://github.com/noblebarkrr/mvsepless -b beta
!pip install uv
!cd mvsepless && uv pip install --no-cache-dir -qq -r requirements.txt
!cd mvsepless && uv pip install --no-cache-dir -qq ./fixed/audio_separator-0.32.0-py3-none-any.whl

# MVSepLess Web-UI

In [ ]:
import os
import re
import time
import urllib
import ipywidgets as widgets
from IPython.display import display, Javascript
import json
import socket
import psutil
import signal
import random
import string
%cd /content/mvsepless

def generate_subdomain(length=8):
    """Генерация случайного субдомена заданной длины"""
    chars = string.ascii_lowercase + string.digits
    return ''.join(random.choice(chars) for _ in range(length))

def kill_process_using_port(port):
    """Находит и завершает процесс, использующий указанный порт"""
    for proc in psutil.process_iter(['pid', 'name', 'connections']):
        try:
            for conn in proc.connections():
                if conn.status == 'LISTEN' and conn.laddr.port == port:
                    print(f"Найден процесс {proc.name()} (PID: {proc.pid}), использующий порт {port}")
                    os.kill(proc.pid, signal.SIGTERM)
                    print(f"Процесс {proc.pid} завершен")
                    time.sleep(2)  # Даем время для освобождения порта
                    return True
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            continue
    return False

# @markdown ## Запуск интерфейса
# @markdown  ---
# @markdown  ### Основные настройки веб интерфейса
port = 7860
# @markdown * Способы поделится приложением

# @markdown > Свойства \ Способ | ngrok | gradio | localtunnel
# @markdown > --- | --- | --- | ---
# @markdown > Требуется VPN | Да | Да | Нет
# @markdown > Стабильность | Высокая | Средняя | Низкая
# @markdown > Срок жизни ссылки | Бесконечный | 1 неделя | Пока субдомен не будет перехвачен
# @markdown <br>




share_method = "gradio" # @param ["ngrok","localtunnel","gradio"]
# @markdown * Язык
language = "ru" # @param ["ru","en"]
# @markdown ---
# @markdown ### Дополнительные настройки
# @markdown * Префикс субдомена для localtunnnel (Если пусто, то используется случайный субдомен по умолчанию)
lt_sub_domain = "mvsepless" # @param {"type":"string"}
# @markdown * Токен авторизации для ngrok\
# @markdown <small> Откуда можно взять токен (только с VPN в РФ): https://dashboard.ngrok.com/get-started/your-authtoken
ngrok_token = "" # @param {"type":"string"}
# @markdown * Путь к файлу шрифта в интерфейсе (.ttf, .otf, .woff, .eot)
font = "" # @param {"type":"string"}


# Проверяем занят ли порт и завершаем процесс
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    if s.connect_ex(('localhost', port)) == 0:
        print(f"Порт {port} занят, пытаемся освободить...")
        if kill_process_using_port(port):
            print(f"Порт {port} успешно освобожден")
        else:
            print(f"Не удалось найти процесс, использующий порт {port}")
            print("Попробуйте другой порт или подождите несколько минут")
            exit()

if share_method == "localtunnel":
    os.system("npm install -g localtunnel &>/dev/null")
    time.sleep(7)
    with open('url.txt', 'w') as file:
        file.write('')
    subdomain = f"{re.sub(r'[^a-zA-Z0-9]', '', lt_sub_domain)}-{generate_subdomain(25)}"


    get_ipython().system_raw(
        f'lt --port {port} '
        f'{f"--subdomain {subdomain}" if lt_sub_domain != "" or lt_sub_domain.isspace() == False else ""} '
        '>> url.txt 2>&1 &'
    )
    time.sleep(3)
    try:
        endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip()
        with open('url.txt', 'r') as file:
            tunnel_url = file.read().replace("your url is: ", "").strip()
        print(f"Публичная ссылка: \033[93m{tunnel_url}\033[0m")
        print("НИКОМУ НЕ ПОКАЗЫВАЙТЕ ЭТУ ССЫЛКУ, А ТО МОГУТ ПЕРЕХВАТИТЬ СУБДОМЕН")

        text_field = widgets.Text(
            value=endpoint_ip,
            description='Пароль:',
            disabled=True
        )
        text_field.add_class("copy-enabled")

        display(text_field)
        display(Javascript("""
        setTimeout(() => {
            const input = document.querySelector('.copy-enabled input');
            if (!input) return;
            const btn = document.createElement('button');
            btn.innerHTML = '📋';
            btn.style.cssText = `
                margin-left: 8px;
                border: none;
                background: none;
                cursor: pointer;
                font-size: 1.2em;
            `;
            input.parentNode.appendChild(btn);
            btn.addEventListener('click', () => {
                navigator.clipboard.writeText(input);
                btn.innerHTML = '✓';
                setTimeout(() => btn.innerHTML = '📋', 2000);
            });
        }, 300);
        """))

    except Exception as e:
        print(f"Ошибка при старте localtunnel: {e}")

cmd_parts = [
    f"python app.py --lang {language} --port {port}",
    f" --share" if share_method == "gradio" else "",
    f" --ngrok_token {ngrok_token}" if share_method == "ngrok" else "",
    f" --font {font}" if font != "" else ""
]
cmd = "".join(cmd_parts)
print(f"Запускаем команду: {cmd}")
!{cmd}

# MVSepLess CLI

In [ ]:
#@title Инференс
#@markdown ---
#@markdown ### Входные данные
#@markdown * Путь к входной папке/файлу:
input_dir = "" # @param {type:"string"}
#@markdown ---
#@markdown ### Выбор модели
#@markdown * Тип модели:
model_type = "bs_roformer" # @param ["mel_band_roformer", "bs_roformer", "mdx23c", "scnet", "htdemucs", "bandit", "bandit_v2", "vr", "mdx"]
#@markdown * Название модели:
model_name = "BS-Roformer_SW" # @param {"type":"string"}
#@markdown ---
#@markdown ### Настройки раздления
# @markdown * Агрессивность для VR ARCH моделей
vr_arch_aggressive = 5 # @param {"type":"slider","min":1,"max":100,"step":1}
# @markdown * Извлечь  инструментал:
instrumental = False # @param {type:"boolean"}
#@markdown ---
#@markdown ### Выходные данные
#@markdown * Формат:
output_format = "mp3" # @param ["mp3", "wav", "flac", "ogg", "opus", "m4a", "aac", "aiff"]
# @markdown * Битрейт
bitrate = 320 # @param {"type":"slider","min":32,"max":320,"step":1}
# @markdown * Выбрать выходные стемы(через пробел, например ("vocal"  "instrumental")):
stems_to_extract = "" # @param {type:"string"}
#@markdown * Шаблон именования выходных файлов:
output_template = "NAME (STEM) MODEL" # @param {type:"string"}
#@markdown * Путь к выходной папке:
output_dir = "/content/output" # @param {type:"string"}
# @markdown * Показать список выходных файлов
show_list_outputs = True # @param {"type":"boolean"}

%cd /content/mvsepless

cmd = [
    "python multi_inference.py separate",
    f"-i \"{input_dir}\"",
    f"-o \"{output_dir}\"",
    f"-mt \"{model_type}\"",
    f"-mn \"{model_name}\"",
    f"-of \"{output_format}\"",
    f"-bitrate \"{bitrate}k\"",
    f"-vr_aggr {vr_arch_aggressive}",
    f"--template \"{output_template}\""
]

if instrumental:
    cmd.append("--instrumental")

if show_list_outputs:
    cmd.append("-l_out")

if stems_to_extract:
    cmd.append(f"--stems {stems_to_extract}")

!{" ".join(cmd)}

/content/mvsepless
Starting inference: bs_roformer/BS-Roformer_SW, bitrate=126k, method=cli, stems=[]
Model already downloaded
Разделение выполняется на ядрах CUDA. Для выполнения на процессоре установите force_cpu=True.
Используется устройство: cuda:0
/usr/local/lib/python3.11/dist-packages/rotary_embedding_torch/rotary_embedding_torch.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.11/dist-packages/rotary_embedding_torch/rotary_embedding_torch.py:268: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
Выбранный чекпоинт: /content/mvsepless/separator/models_cache/bs_roformer/BS-Roformer_SW.ckpt
Стемы: ['bass', 'drums', 'other', 'vocals', 'guitar', 'piano']
Потрачено времени на загрузку модели: 5.74 сек.
Выбранное аудио: /content/Galileo_call_Instrumental

In [3]:
#@title Показать список моделей
filter = "" # @param {"type":"string"}
%cd /content/mvsepless
cmd = f"python multi_inference.py list {f' -l_filter {filter}' if filter != '' else ''}"
!$cmd

/content/mvsepless

Available Models Information:

Model Type: MEL_BAND_ROFORMER
--------------------------------------------------
+---------------------------------------------------------+--------------+---------------------+----------------+
| Model Name                                              | Stems        | Target Instrument   | Primary Stem   |
+=========================================================+==============+=====================+================+
| Mel-Band-Roformer_Vocals_kimberley_jensen               | vocals       | vocals              | N/A            |
|                                                         | other        |                     |                |
+---------------------------------------------------------+--------------+---------------------+----------------+
| Mel-Band-Roformer_InstVoc_Duality_v1_unwa               | Vocals       |                     | N/A            |
|                                                         | Instrument